In [10]:
#check this -> https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical
import keras
import keras.optimizers
import matplotlib.pyplot as plt

tf.__version__

'1.12.0'

In [11]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#scale define, makes easier for network to learn
#x_train = tf.keras.utils.normalize(x_train, axis=1)
#x_test = tf.keras.utils.normalize(x_test, axis=1)
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
#y_train[0]


In [14]:
model = Sequential()
#args: (filters, window size, input_shape)
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
#model.add(Activation("relu"))
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
#model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, kernel_size=(3,3), activation='relu'))
#model.add(Activation("relu"))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))
#model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='categorical_crossentropy', 
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=12, batch_size=128, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 93s 2ms/step - loss: 0.5914 - acc: 0.8454 - val_loss: 0.0715 - val_acc: 0.9777
Epoch 2/12
60000/60000 [==============================] - 84s 1ms/step - loss: 0.1355 - acc: 0.9598 - val_loss: 0.0530 - val_acc: 0.9828
Epoch 3/12
60000/60000 [==============================] - 84s 1ms/step - loss: 0.1042 - acc: 0.9690 - val_loss: 0.0458 - val_acc: 0.9852
Epoch 4/12
60000/60000 [==============================] - 85s 1ms/step - loss: 0.0867 - acc: 0.9746 - val_loss: 0.0433 - val_acc: 0.9862
Epoch 5/12
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0784 - acc: 0.9768 - val_loss: 0.0413 - val_acc: 0.9877
Epoch 6/12
60000/60000 [==============================] - 82s 1ms/step - loss: 0.0677 - acc: 0.9792 - val_loss: 0.0378 - val_acc: 0.9877
Epoch 7/12
60000/60000 [==============================] - 88s 1ms/step - loss: 0.0649 - acc: 0.9806 - val_loss: 0.0392 - val_acc

In [15]:
model.save('9852.model')